In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import jax_smi
jax_smi.initialise_tracking()

In [6]:
from scripts.train_gated_sae import main
main()

Loading model...


/home/neverix/.cache/pypoetry/virtualenvs/saex-U2at97x7-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at gpt2 were not used when initializing FlaxGPT2LMHeadModel: {('h', '4', 'attn', 'bias'), ('h', '2', 'attn', 'bias'), ('h', '10', 'attn', 'bias'), ('h', '9', 'attn', 'bias'), ('h', '8', 'attn', 'bias'), ('h', '1', 'attn', 'bias'), ('h', '7', 'attn', 'bias'), ('h', '5', 'attn', 'bias'), ('h', '3', 'attn', 'bias'), ('h', '0', 'attn', 'bias'), ('h', '11', 'attn', 'bias'), ('h', '6', 'attn', 'bias')}
- This IS expected if you are initializing FlaxGPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraini

Creating buffer...
Creating SAE...
Loading dataset...


/home/neverix/.cache/pypoetry/virtualenvs/saex-U2at97x7-py3.12/lib/python3.12/site-packages/datasets/load.py:1486: FutureWarning: The repository for Skylion007/openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Skylion007/openwebtext
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Exception in thread Thread-16 (inner):
Traceback (most recent call last):
  File "/home/neverix/.pyenv/versions/3.12.3/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/home/neverix/.cache/pypoetry/virtualenvs/saex-U2at97x7-py3.12/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/home/neverix/.pyenv/versions/3.12.3/lib/python3.12/threading.

Training for 100000 iterations
Sparsity coefficient: 5e-05


Learning rate: 0.0004 warmed up for 128 iterations and cycled every 100000 iterations


  0%|          | 0/100000 [00:00<?, ?it/s]

In [ ]:
from scripts.train_phi_sae import main
main()

/home/neverix/.cache/pypoetry/virtualenvs/saex-U2at97x7-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading model...
Creating buffer...
Creating SAE...
Loading dataset...


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Training for 100000 iterations
Sparsity coefficient: 1e-06


wandb: Currently logged in as: neverix. Use `wandb login --relogin` to force relogin


Learning rate: 0.0004 warmed up for 128 iterations and cycled every 100000 iterations


  0%|          | 0/100000 [00:00<?, ?it/s]

Exception in thread Thread-4 (inner):
Traceback (most recent call last):
  File "/home/neverix/.pyenv/versions/3.12.3/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/home/neverix/.cache/pypoetry/virtualenvs/saex-U2at97x7-py3.12/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/home/neverix/.pyenv/versions/3.12.3/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/home/neverix/.cache/pypoetry/virtualenvs/saex-U2at97x7-py3.12/lib/python3.12/site-packages/jax_smi/initialise_tracking.py", line 10, in inner
    posix.rename(f'{dir_prefix}/memory.prof.new', f'{dir_prefix}/memory.prof')  # atomic
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: '/dev/shm/memory.prof.new' -> '/dev/shm/memory.prof'


[3380]
[3380]
[3380]
[3380]
[3380]
[3380]
[3380]
[3380]
[3380]
[3380]
[3380]
[3380]
[3380]
[3380]
[3380]
[3380]
[6240]
[6240]
[6240]
[6240]
[6240]
[6240]
[6240]
[6240]
[6240]
[6240]
[6240]
[6240]
[6240]
[6240]
[6240]
[6240]
[8357]
[8357]
[8357]
[8357]
[8357]
[8357]
[8357]
[8357]
[8357]
[8357]
[8357]
[8357]
[8357]
[8357]
[8357]
[8357]
[11078]
[11078]
[11078]
[11078]
[11078]
[11078]
[11078]
[11078]
[11078]
[11078]
[11078]
[11078]
[11078]
[11078]
[11078]
[11078]
[12254]
[12254]
[12254]
[12254]
[12254]
[12254]
[12254]
[12254]
[12254]
[12254]
[12254]
[12254]
[12254]
[12254]
[12254]
[12254]
[15430]
[15430]
[15430]
[15430]
[15430]
[15430]
[15430]
[15430]
[15430]
[15430]
[15430]
[15430]
[15430]
[15430]
[15430]
[15430]
[2564]
[2564]
[2564]
[2564]
[2564]
[2564]
[2564]
[2564]
[2564]
[2564]
[2564]
[2564]
[2564]
[2564]
[2564]
[2564]
[4342]
[4342]
[4342]
[4342]
[4342]
[4342]
[4342]
[4342]
[4342]
[4342]
[4342]
[4342]
[4342]
[4342]
[4342]
[4342]
[6383]
[6383]
[6383]
[6383]
[6383]
[6383]
[6383]
[6383]


dead,▁▁▁▂▂▁▄▁▃▃▂▁▁▁▁▂▁▁▂▁▁▂▁▁▁▁▁█▇▄▂▃▃▁▂▂▂▂▁▂
l0,▅▃▃▂▂▂▂▃▂▁▁▃▃▄▄▄▄▄▄▄▄▄▄▆▆▅▆▁▁▆▅▆▆▅▇███▆█
loss,▆█▃▃▄▄▃▃▂▁▁▇▆▃▆▄▄▄▄▅▄▄▅▄▅▄▃▁▁▃▄▃▃▃▆▃▃▄▃▄
loss_death,▁▁▁▄▄▃▇▁▅▃▂▁▄▁▁▃▂▂▃▁▂▄▃▁▃▂▁█▆█▃▅▅▁▅▂▃▃▂▃
loss_reconstruction,▅█▃▂▄▄▃▃▂▁▁▇▆▃▆▄▃▃▃▄▃▄▅▃▅▄▂▁▁▂▃▂▂▃▅▂▂▃▂▃
loss_sparsity,▅▃▃▂▂▂▂▃▂▁▁▃▄▃▄▄▄▄▄▅▅▅▅▆▆▆▅▁▁▅▅▆▅▅█▅▆▇▆▇
max_time_since_fired,▁▂▂▂▂▂▂▂▂▃▂▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂▄█▃▆▂▄▂▂▂▂▂▃▂
tokens_processed,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
var_explained,▆▁▆▇▇▇▇▇█▇▇▇▇▆▇████████████▆▇███████████
dead,0.00122
l0,453.95856


In [ ]:
from scripts.train_gpt2_sae import main
main(restore="weights/gpt2-20-base.safetensors", layer=32)
# python -m scripts.train_gpt2_sae --save_steps 0 --is_xl=False --layer=9
# main(save_steps=0, is_xl=False, layer=1)

Loading model...


/home/neverix/.cache/pypoetry/virtualenvs/saex-U2at97x7-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Some weights of the model checkpoint at openai-community/gpt2-xl were not used when initializing FlaxGPT2LMHeadModel: {('h', '27', 'attn', 'bias'), ('h', '17', 'attn', 'bias'), ('h', '15', 'attn', 'bias'), ('h', '36', 'attn', 'bias'), ('h', '18', 'attn', 'bias'), ('h', '44', 'attn', 'bias'), ('h', '19', 'attn', 'bias'), ('h', '14', 'attn', 'bias'), ('h', '34', 'attn', 'bias'), ('h', '39', 'attn', 'bias'), ('h', '21', 'attn', 'bias'), ('h', '32', 'attn', 'bias'), ('h', '33', 'attn', 'bias'), ('h', '8', 'attn', 'bias'), ('h', '6', 'attn', 'bias'), ('h', '9', 'attn', 'bias'), ('h', '29', 'attn', 'bias'), ('h', '16', 'attn', 'bias'), ('h', '12', 'attn', 'bias'), ('h', '13', 'attn', 'bias'), ('h', '30', 'attn', 'bias'), ('h', '35', 'attn', 'bias'), ('h', '40', 'attn', 'bias'), ('h', '7', 'attn', 'bias'), ('h', '4', 'attn', 'bias'), ('h', '20', 'attn', 'bias'), ('h', '43', 'attn', 'bias'), ('h', '11', 'attn', 'bias'), ('h', '5', 'attn', 'bias'), ('h', '22', 'attn', 'bias'), ('h', '10', 'attn

Creating buffer...
Creating SAE...
Loading checkpoint (weights/gpt2-20-base.safetensors)...
Loading dataset...


/home/neverix/.cache/pypoetry/virtualenvs/saex-U2at97x7-py3.12/lib/python3.12/site-packages/datasets/load.py:1486: FutureWarning: The repository for Skylion007/openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Skylion007/openwebtext
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Training for 100000 iterations
Sparsity coefficient: 5e-05


Learning rate: 0.0004 warmed up for 128 iterations and cycled every 100000 iterations


  0%|          | 0/100000 [00:00<?, ?it/s]

: 